In [ ]:
#imports
from __future__ import division, print_function, absolute_import
import matplotlib.pyplot as plt
# %matplotlib inline
import numpy as np
import tensorflow as tf
import random, math
import sys, os, time, json
import scipy.interpolate as interp 
import csv 
import copy
import timeit
import pandas as pd
import re
import pdb
import traceback
from tqdm import tqdm
import glob

from sklearn.model_selection import train_test_split
from sklearn.preprocessing import MinMaxScaler

os.environ['CUDA_VISIBLE_DEVICES'] = "0"

In [ ]:
def load_data(lab_value, run_num = 0, file_dict=None):
#     ecg_directory = '/gunvant_nas_trop/ecgs/%s' % lab_value
#     file_dict = {}
#     fail = []
    
#     for filename in tqdm(os.listdir(ecg_directory)):
#         name = filename[:-4]
#         if name not in file_dict.keys():
#             try:
# #                 pdb.set_trace()
#                 file_dict[name] = [-1,1]
#                 ecg_raw = np.loadtxt(ecg_directory + '/' + filename, skiprows=1, delimiter=',', usecols=range(12))
#                 len_leads = len(ecg_raw)
#                 if len_leads == 5000:
#                     file_dict[name][0] = ecg_raw[::2]
#                 elif len_leads == 2500:
#                     file_dict[name][0] = ecg_raw
#                 else:
#                     print('len_leads:', len_leads)
#             except:
#                 fail.append(filename)
# #         if len(file_dict)>100:
# #             break;

#     print('Failed to find:', fail)
#     print('Num ECGS extracted:',len(file_dict))
#     np.save('trop_all_v3_dict.npy',file_dict)
    
    if file_dict is None:
        file_dict = np.load('mvp.npy',allow_pickle=True,encoding='latin1')
    #     file_dict = np.load('trop_catLh_dict.npy',allow_pickle=True,encoding='latin1')
    #     file_dict = np.load('k_ser_all_dict.npy',allow_pickle=True,encoding='latin1')
    #     file_dict = np.load('potassium_serum_dict.npy',allow_pickle=True,encoding='latin1')
        file_dict = file_dict.item()

    train_voltages = []
    train_labels = []
    val_voltages = []
    val_labels = []
    test_voltages = []
    test_labels = []
    
    train_labels_raw = []
    val_labels_raw = []
    test_labels_raw = []
#     min_max_scaler = MinMaxScaler()
#     quantile_scaler = QuantileTransformer()
    
    lab_values = pd.read_csv("cve_uuid_mrn.csv")
#     lab_values = pd.read_csv("4h_nearest_%s_uuid.csv" % lab_value)
#     lab_values = pd.read_csv("6h_back_%s_uuid.csv" % lab_value)

#     lab_values_exclude = pd.read_csv("4h_nearest_trop_all_uuid.csv")
#     uuids_exclude = lab_values_exclude['UUID'].drop_duplicates()
    
#     lab_values = lab_values.sort_values(['UUID','result_time']).drop_duplicates(subset='UUID',keep='last') #done already
#     lab_values['norm2'] = min_max_scaler.fit_transform(lab_values['ord_value'].values.reshape(-1, 1))
#     lab_values['norm2'] = lab_values['ord_value']
#     min_max_scaler.fit(lab_values['ord_value'].values.reshape(-1, 1))  

    mrns = lab_values['mrn'].drop_duplicates()
    lab_values = lab_values.set_index('UUID')
    mrn_train,mrn_not_train = train_test_split(mrns,test_size=0.3, random_state= 4* (run_num+1))
    mrn_val, mrn_test = train_test_split(mrn_not_train,test_size=0.5, random_state=4* (run_num+1))
    
    CUTOFF = 0.01
    train_dems = [] # [gender, age, weight, height, bsa]
    val_dems = []
    test_dems = []
    
    
    failed = []
    excluded = []
    for mrn in tqdm(mrn_train.values):
        for key in lab_values[lab_values['mrn'] == mrn].index:
            try:
                label = float(lab_values.loc[key,'disease'])
                this_ecg = file_dict[key][0]
                if label>=0.0 and np.max(this_ecg)<15000 and np.min(this_ecg)>-15000:# and key not in uuids_exclude.values:

                    train_voltages.append(this_ecg)

                    if label < CUTOFF:
                        train_labels.append([0])
                    else:
                        train_labels.append([1])

                    train_labels_raw.append([label])
    #                 train_labels.append(this_hr)
                else:
                    excluded.append(key)
                    pass
            except:
                failed.append(key)
                pass
    
    for mrn in tqdm(mrn_val.values):
        for key in lab_values[lab_values['mrn'] == mrn].index:
            try:
                label = float(lab_values.loc[key,'disease'])
    #             this_hr = get_hr(file_dict[key][0])
    #             if not math.isnan(this_hr): 
                this_ecg = file_dict[key][0]
                if label>=0.0 and np.max(this_ecg)<15000 and np.min(this_ecg)>-15000:# and key not in uuids_exclude.values:


                    val_voltages.append(file_dict[key][0])

                    if label < CUTOFF:
                        val_labels.append([0])
                    else:
                        val_labels.append([1])
                    val_labels_raw.append([label])
    #                 val_labels.append(this_hr)
                else:
                    excluded.append(key)
                    pass
            except:
                failed.append(key)
                pass

    for mrn in tqdm(mrn_test.values):
        for key in lab_values[lab_values['mrn'] == mrn].index:
            try:
                label = float(lab_values.loc[key,'disease'])
    #             this_hr = get_hr(file_dict[key][0])
    #             if not math.isnan(this_hr):
                this_ecg = file_dict[key][0]
                if label>=0.0 and np.max(this_ecg)<15000 and np.min(this_ecg)>-15000:# and key not in uuids_exclude.values:
                    test_voltages.append(file_dict[key][0])

                    if label < CUTOFF:
                        test_labels.append([0])
                    else:
                        test_labels.append([1])

                    test_labels_raw.append([label])
    #                 test_labels.append(this_hr)
                else:
                    excluded.append(key)
    #                     pdb.set_trace() 
                    pass
            except:
                failed.append(key)
                pass
    
    print('Excluded:', len(excluded))

    x_train = np.array(train_voltages)
    y_train = np.array(train_labels)

    x_val = np.array(val_voltages)

    y_val = np.array(val_labels)
#     y_val = (np.array(val_labels).reshape((-1,1))  - labels_min) / (labels_max - labels_min)
#     y_val = (np.array(val_labels).reshape((-1,1))  - labels_mean) / (labels_std)

#     x_test = np.swapaxes(test_voltages,1,2)
#     x_test = ( ( np.clip(test_voltages, global_min, global_max) - global_min ) / (global_max - global_min) ).reshape((-1,2500,12))
    x_test = np.array(test_voltages)

    y_test = np.array(test_labels)

    print(x_train.shape, y_train.shape, x_val.shape, y_val.shape, x_test.shape, y_test.shape, np.array(train_dems).shape, np.array(val_dems).shape, np.array(test_dems).shape)
    return x_train, x_val, x_test, y_train, y_val, y_test, np.array(train_dems), np.array(val_dems), np.array(test_dems), train_labels_raw, val_labels_raw, test_labels_raw

# def get_hr(ecg_raw):
#     hrs = []
#     _,_,_,_,_,_,hrs = ecg.ecg(signal=ecg_raw[:,1], sampling_rate=250, show=False)
    
#     return np.mean(hrs)    

In [ ]:
x_train, x_val, x_test, y_train, y_val, y_test, dems_train, dems_val, dems_test, train_labels_raw, val_labels_raw, test_labels_raw = load_data('_')

In [ ]:
import numpy as np
import keras

class ValidDataGenerator(keras.utils.Sequence):
    'Generates data for Keras'
    def __init__(self, ecgs, labels, dems=None, batch_size=32, dim=2048, n_channels=12,
                 n_classes=60, shuffle=False, train_cats = None):
        'Initialization'
        self.dim = dim
        self.batch_size = batch_size
        self.labels = labels
        self.ecgs = ecgs
        self.ecg_inds = np.arange(ecgs.shape[0])
        self.n_channels = n_channels
        self.n_classes = n_classes
        self.shuffle = shuffle
        self.train_cats = train_cats
        self.dems = dems
        self.on_epoch_end()

    def __len__(self):
        'Denotes the number of batches per epoch'
        return int(np.floor(self.ecgs.shape[0] / self.batch_size))

    def __getitem__(self, index):
        'Generate one batch of data'
        # Generate indexes of the batch
        indexes = self.ecg_inds[index*self.batch_size:(index+1)*self.batch_size]

        # Generate data
        X, y = self.__data_generation(indexes)

        return X, y

    def on_epoch_end(self):
        'Updates indexes after each epoch'
        self.ecg_inds = np.arange(self.ecgs.shape[0])
        if self.shuffle == True:
            np.random.shuffle(self.ecg_inds)

    def __data_generation(self, indexes):
        'Generates data containing batch_size samples' # X : (n_samples, *dim, n_channels)
#         # Initialization
#         X = np.empty((self.batch_size, *self.dim, self.n_channels))
#         y = np.empty((self.batch_size, self.n_classes), dtype=float32)

        X = self.ecgs[indexes,:self.dim,:]
        y = self.labels[indexes, :]
#         d = self.dems[indexes,:]

#         return [np.array(X),np.array(d)], y
        return X, y

In [ ]:
import random
from scipy.ndimage import interpolation

class TrainDataGenerator(keras.utils.Sequence):
    'Generates data for Keras'
    def __init__(self, ecgs, labels, dems=None, batch_size=32, dim=2048, n_channels=12,
                 n_classes=60, shuffle=True, train_cats = None, aug_rescale=(0.85, 1.15)):
        'Initialization'
        self.dim = dim
        self.batch_size = batch_size
        self.labels = labels
        self.ecgs = ecgs
        self.ecg_inds = np.arange(ecgs.shape[0])
        self.n_channels = n_channels
        self.n_classes = n_classes
        self.shuffle = shuffle
        
        
        self.aug_rescale = aug_rescale
        
        self.train_cats = train_cats
        
        self.dems = dems
        self.calc_sampling_weights()

    def __len__(self):
        'Denotes the number of batches per epoch'
        return int(np.floor(self.ecgs.shape[0] / self.batch_size))

    def __getitem__(self, index):
        'Generate one batch of data'
        # Generate indexes of the batch
        
        indexes = np.random.choice(self.ecg_inds, size=self.batch_size, replace=False, p=self.ind_weights)
        
#         indexes = self.ecg_inds[index*self.batch_size:(index+1)*self.batch_size]

        # Generate data
#         pdb.set_trace()
        
        X, y = self.__data_generation(indexes)

        return X, y
    
    def calc_sampling_weights(self):
        'Initially, calculates sampling weights for each training item'
        
        cats, cat_counts = np.unique(self.train_cats,return_counts=True)
        weights = np.zeros(len(cats))
        for xx in range(len(cats)):
            weights[xx] = 1/ ( cat_counts[xx]/len(self.train_cats)  )
        
        tot_weight = sum([weights[xx] * cat_counts[xx] for xx in range(len(weights))])
        scaled_weights = []
        for weight in weights:
            scaled_weights.append(weight/tot_weight)
        
        ind_weights = np.zeros(self.ecgs.shape[0])
        for ii, val in enumerate(self.train_cats):
            ind_weights[ii] = scaled_weights[val]
        
        print('Weights', weights)
        print('Scaled_weights',scaled_weights)
        
        
        
        self.ind_weights = ind_weights
    
    def augment(self, X_in):
        'Randomly rescales input ecg and return random window of size self.dim'
        
        scale_f = random.random() * (self.aug_rescale[1] - self.aug_rescale[0]) + self.aug_rescale[0]
#         print(scale_f)
        X_new = interpolation.zoom(X_in,zoom=[scale_f,1])
        start = random.randrange(0, X_new.shape[0]-self.dim+1)
        end = start + self.dim
        
        return X_new[start:end, :]

    def __data_generation(self, indexes):
        'Generates data containing batch_size samples' # X : (n_samples, *dim, n_channels)
#         # Initialization
#         X = np.empty((self.batch_size, *self.dim, self.n_channels))
#         y = np.empty((self.batch_size, self.n_classes), dtype=float32)

        X = self.ecgs[indexes,:,:]
        X_out = []
        for X_in in X:
            X_out.append(self.augment(X_in))
#             X_out.append(X_in[:self.dim])
        
        X_out = np.array(X_out)
        y = self.labels[indexes, :]
        
#         d = []
#         for each in indexes:
#             d.append(self.dems[each,:])
#         d = self.dems[indexes,:]
#         print(d.shape)
        
#         pdb.set_trace()
        
#         return [np.array(X),np.array(d)], y
        return X_out, y

In [ ]:
from keras import backend as K
from keras.layers import Layer
from keras import initializers, regularizers, constraints
from keras.layers.normalization import BatchNormalization
from keras import regularizers

from sklearn.metrics import f1_score, confusion_matrix, roc_curve, auc
from keras.callbacks import Callback


def f1_score(y_true, y_pred):

    # Count positive samples.
    c1 = K.sum(K.round(K.clip(y_true * y_pred, 0, 1)))
    c2 = K.sum(K.round(K.clip(y_pred, 0, 1)))
    c3 = K.sum(K.round(K.clip(y_true, 0, 1)))

    # How many selected items are relevant?
    precision = c1 / (c2 + K.epsilon())

    # How many relevant items are selected?
    recall = c1 / (c3 + K.epsilon())

    # Calculate f1_score
    f1_score = 2 * (precision * recall) / (precision + recall + K.epsilon())
    return f1_score 

class IntervalEvaluation(Callback):
    def __init__(self, validation_data=(), log_f = 'log.csv', interval=10):
        super(Callback, self).__init__()

        self.interval = interval
        self.X_val, self.y_val = validation_data
        self.log_f = log_f

    def on_epoch_end(self, epoch, logs={}):
        if epoch % self.interval == 0:
            y_pred = self.model.predict(self.X_val, verbose=0)
            Y_val = np.round(self.y_val)
            Y_pred = np.round(y_pred)
#             print(Y_pred)
            cm = confusion_matrix(Y_val, Y_pred)
            cm = cm.astype('float') / cm.sum(axis=1)[:, np.newaxis]
            
            fpr, tpr, _ = roc_curve(self.y_val, y_pred)
            roc_auc = auc(fpr, tpr)
            
            print("interval evaluation - epoch: {:d} - scores:".format(epoch), cm.diagonal(), roc_auc)
            
            with open(self.log_f,'a') as f:
                f.write("epoch: %d - accs: %s - auc: %f\n" % (epoch+1, ', '.join(str(x) for x in cm.diagonal()), roc_auc) )


def build_unet(**params): # not a unet this is our modified 12 lead weston network
    import keras
    from keras.layers import Conv1D, BatchNormalization, Add, MaxPooling1D
    from keras.optimizers import Adam
    from keras.layers.core import Dense, Activation, Flatten
    from keras.models import Model
    from keras.layers import Input
    from keras.layers import Dropout
    
    
    inputs = Input(shape=params['input_shape'],
                   dtype='float32',
                   name='inputs')
    
    r = lambda: keras.regularizers.l2(params["l2_kern_weight"])

    layer = Conv1D(
             filters=params["conv_num_filters_start"],
             kernel_size=params["conv_filter_length"],
             strides=1,
             padding='same', 
             kernel_regularizer=r(), 
            # weston does not use kernel initalizer 
             kernel_initializer=params["conv_init"])(inputs)
    
    layer = BatchNormalization()(layer)
    layer = Activation(params["conv_activation"])(layer)

    layer2 = Conv1D(
             filters=params["conv_num_filters_start"],
             kernel_size=params["conv_filter_length"],
             strides=1,
             padding='same', 
             kernel_regularizer=r(), 
            # weston does not use kernel initalizer 
             kernel_initializer=params["conv_init"])(layer)

    layer2 = BatchNormalization()(layer2)
    layer2 = Activation(params["conv_activation"])(layer2)
    layer2 = Dropout(params["conv_dropout"])(layer2)
    layer2 = Conv1D(
                 filters=params["conv_num_filters_start"],
                 kernel_size=params["conv_filter_length"],
                 strides=1,
                 padding='same', 
                 kernel_regularizer=r(), 
                # weston does not use kernel initalizer 
                 kernel_initializer=params["conv_init"])(layer2)
    
    layer = Add()([layer,layer2])
    # x = tf.layers.max_pooling1d(x, pool_size=2, strides=2, padding='same')
    layer = MaxPooling1D(pool_size=2, strides=2,padding='same')(layer)
    
    for i in range(1, 1 + params["num_middle_layers"]):
        layer2 = layer
        n_filters = params["conv_num_filters_start"] * 2 ** (i // params["conv_increase_channels_at"])

        for j in range(params["num_convs_per_layer"]):
            layer2 = BatchNormalization()(layer2)
            layer2 = Activation(params["conv_activation"])(layer2)
            layer2 = Dropout(params["conv_dropout"])(layer2)
            layer2 = Conv1D(
                 filters=n_filters,
                 kernel_size=params["conv_filter_length"],
                 strides=1,
                 padding='same', 
                 kernel_regularizer=r(), 
                 kernel_initializer=params["conv_init"])(layer2)
            
        if i % params["conv_increase_channels_at"] == 0:
            layer = layer2
        else:
            layer = Add()([layer,layer2])

        if i % params["conv_pool_at"] == 0:
            # add padding = 'same'
            layer = MaxPooling1D(pool_size=2,strides=2,padding='same')(layer)

    layer = BatchNormalization()(layer)
    layer = Activation(params["conv_activation"])(layer)
    layer = Flatten()(layer)
    for i in range(params["hidden_layers"]):
        layer = Dense(params["hidden_size"])(layer)

    layer = Dense(params["num_categories"])(layer)
    
    output = Activation("sigmoid")(layer)
    
    model = Model(inputs=[inputs], outputs=[output])

    optimizer = Adam(
        lr=params["learning_rate"],
        clipnorm=params.get("clipnorm", 1))

    model.compile(loss='binary_crossentropy',
                  optimizer=optimizer,
                  metrics=['acc', f1_score])
    
    return model

def get_filename_for_saving(save_dir, run):
    return os.path.join(save_dir,
            "{val_loss:.3f}-{epoch:03d}-{loss:.3f}-%d.hdf5" % run)

In [ ]:
all_hists = {}

rt_dir = '/volume/12_lead/models/cve_binary_sean/baseline'
if not os.path.isdir(rt_dir):
    print('Making: %s', rt_dir)
    os.mkdir(rt_dir)

for iii in range(1):
    x_train, x_val, x_test, y_train, y_val, y_test, dems_train, dems_val, dems_test, train_labels_raw, val_labels_raw, test_labels_raw = load_data('trop_all', iii)
    K.clear_session()
    params={
        "conv_subsample_lengths": [1, 2, 1, 2, 1, 2, 1, 2, 1, 2, 1, 2, 1, 2, 1], #15 layers
        "conv_filter_length": 16,
        "conv_num_filters_start": 64,
        "conv_init": "he_normal",
        "conv_activation": "relu",
        "conv_dropout": 0.2,
        "conv_num_skip": 2,
        "conv_increase_channels_at": 4,
        "conv_pool_at": 2,
        "num_categories":1,

        "input_shape":[2048,12],

        "l2_kern_weight": 0.0001,
        "l2_bias_weight": 0.0001,

        "hidden_layers": 1,
        "hidden_size":1024,

        "num_middle_layers": 15,
        "num_convs_per_layer": 1,

        "learning_rate": 1e-3,
        "batch_size": 64,

        "compile": True,

    }
    # assert len(params['conv_subsample_lengths']) == params["conv_filter_length"]

    gen_params = {
        'dim': (2048),
        'batch_size': 128,
        'n_classes': 1,
        'n_channels': 12,
        'shuffle': True,
        'train_cats': y_train.squeeze()
    }


    # # Generators
    training_generator = TrainDataGenerator(x_train, y_train, **gen_params)
    validation_generator = ValidDataGenerator(x_val, y_val, **gen_params)

    # # # Generators
    # training_generator = TrainDataGenerator(x_train[:,:2048,:], y_train, dems_train, **gen_params)
    # validation_generator = ValidDataGenerator(x_val[:,:2048,:], y_val, dems_val, **gen_params)


    model = build_unet(**params)
    print(model.summary())

    stopping = keras.callbacks.EarlyStopping(patience=20)

    reduce_lr = keras.callbacks.ReduceLROnPlateau(
        factor=0.1,
        patience=2,
        min_lr=1e-4 * 0.001)
    
    sv_dir_base = os.path.join(rt_dir, 'run%d' % iii) 
    if not os.path.isdir(sv_dir_base):
        print('Making: %s', sv_dir_base)
        os.mkdir(sv_dir_base)
    
    checkpointer = keras.callbacks.ModelCheckpoint(filepath=get_filename_for_saving(sv_dir_base, iii),
        save_best_only=False)

    ival = IntervalEvaluation(validation_data=(x_val[:,:2048,:], y_val), 
                              log_f = os.path.join(sv_dir_base,'log_binary_1_cutoff_ensemble_evals_run%d.txt' % iii), interval=1)

    csv_logger = keras.callbacks.CSVLogger(os.path.join(sv_dir_base, 'log_binary_1_cutoff_ensemble_run%d.csv' % iii), append=True)
    
    history = model.fit_generator(training_generator,
                        validation_data=validation_generator,
                        epochs=30,
                        callbacks=[checkpointer, reduce_lr, stopping, ival, csv_logger], verbose=2)
    
    all_hists[iii] = history

In [ ]:
y_predicted = model.predict(x_test[:,:2048,:])

In [ ]:
np.save('/models/trop_binary_sean/all_hists_cutoff_0.04.npy',  {key:value.history for key, value in all_hists.items()})

In [ ]:
iii=0
x_train, x_val, x_test, y_train, y_val, y_test, dems_train, dems_val, dems_test, train_labels_raw, val_labels_raw, test_labels_raw, file_dict = load_data('trop_all', iii, file_dict)

# x_train_cath, x_val_cath, x_test_cath, y_train_cath, y_val_cath, y_test_cath, dems_train_cath, dems_val_cath, dems_test_cath, train_labels_raw_cath, val_labels_raw_cath, test_labels_raw_cath, _ = load_cath_data('trop_all', iii, file_dict = None)
# x_cath = np.concatenate( (x_train_cath, x_val_cath, x_test_cath), axis=0)
# y_cath = np.concatenate( (y_train_cath, y_val_cath, y_test_cath), axis=0)

In [ ]:
val_results = {}
val_results['precisions'] = precisions
val_results['recalls'] = recalls
val_results['fscores'] = fscores
val_results['cutoffs'] = cutoffs

test_results = {}
test_results['precisions'] = precisions_test
test_results['recalls'] = recalls_test
test_results['fscores'] = fscores_test
test_results['supports'] = supports_test

print(val_results)
print(test_results)

results_85 = pd.DataFrame({'val_results': val_results, 'test_results': test_results})
results_85.to_pickle('results_85_sensitivity.pickle')

In [ ]:
test_recalls = [x[0] for x in test_results['fscores']]
print( np.mean(test_recalls), np.std(test_recalls) )


In [ ]:
precisions, recalls, fscores, cutoffs = [], [], [], []
precisions_test, recalls_test, fscores_test, supports_test = [], [], [], []
for iii in range(10):
#     iii=0
    x_train, x_val, x_test, y_train, y_val, y_test, dems_train, dems_val, dems_test, train_labels_raw, val_labels_raw, test_labels_raw = load_data('trop_all', iii)

    K.clear_session()

    def f1_score(y_true, y_pred):

        # Count positive samples.
        c1 = K.sum(K.round(K.clip(y_true * y_pred, 0, 1)))
        c2 = K.sum(K.round(K.clip(y_pred, 0, 1)))
        c3 = K.sum(K.round(K.clip(y_true, 0, 1)))

        # If there are no true samples, fix the F1 score at 0.
        if c3 == 0:
            return 0

        # How many selected items are relevant?
        precision = c1 / c2

        # How many relevant items are selected?
        recall = c1 / c3

        # Calculate f1_score
        f1_score = 2 * (precision * recall) / (precision + recall)
        return f1_score 

    models = glob.glob('/volume/12_lead/models/cve_binary_sean/baseline/run%d/*.hdf5' % iii)
    print('Found: ', models)
    # model_1 =  keras.models.load_model("/models/trop_binary_sean/cutoff_0.02/run%d/0.594-022-0.568-9.hdf5" % iii, custom_objects={'f1_score': f1_score})
    model_1 = keras.models.load_model(models[0], custom_objects={'f1_score': f1_score})
    # model_1 =  keras.models.load_model("/models/trop_binary/0.400-019-0.310.hdf5", custom_objects={'f1_score': f1_score, 'Sine': Sine, 'SIRENInitializer': SIRENInitializer})

    # from keras.optimizers import Adam
    # optimizer = Adam(
    #         lr=params["learning_rate"],
    #         clipnorm=params.get("clipnorm", 1))
    # model.compile(loss='binary_crossentropy',
    #                   optimizer=optimizer,
    #                   metrics=['mean_absolute_error'])
    # model_1.summary()
    
    y_predicted = model_1.predict(x_test[:,:2048,:])
    y_predicted_val = model_1.predict(x_val[:,:2048,:])
    precision, recall, fscore, cutoff = process_val(0.85)
    
    cutoffs.append(cutoff)
    precisions.append(precision)
    recalls.append(recall)
    fscores.append(fscore)
    
    print('TEST SET:')
    print(classification_report(y_test, y_predicted>cutoff))
    precision, recall, fscore, support = precision_recall_fscore_support(y_test, y_predicted>cutoff)
    precisions_test.append(precision)
    recalls_test.append(recall)
    fscores_test.append(fscore)
    supports_test.append(support)
    
    
    

In [ ]:
#FOR VAL

def process_val(target_rec = 0.9):
    from sklearn.metrics import f1_score, recall_score, precision_recall_curve, precision_recall_fscore_support
    import matplotlib.pyplot as plt
    
    fpr = dict()
    tpr = dict()
    roc_auc = dict()
    for i in range(1):
        fpr[i], tpr[i], _ = roc_curve(y_val[:, i], y_predicted_val[:, i])
        roc_auc[i] = auc(fpr[i], tpr[i])

    print(roc_auc)

    precision, recall, _ = precision_recall_curve(y_val, y_predicted_val)
#     plt.plot(recall, precision, label='Validation')
#     plt.xlabel('Recall')
#     plt.ylabel('Precision')
#     plt.legend()
#     plt.show()

    scores = []
    for x in range(1000):
        scores.append(f1_score(y_val, (y_predicted_val>x/1000), pos_label=0))
    print(np.max(scores), np.where(scores == np.max(scores))[0][0]/1000)

    scores = []
    recs = []
    for x in range(1000):
        scores.append(f1_score(y_val, (y_predicted_val>x/1000), pos_label=1))
        this_rec = recall_score(y_val, (y_predicted_val>x/1000), pos_label=1)
        recs.append(this_rec)
    print(np.max(scores), np.where(scores == np.max(scores))[0][0]/100, f1_score(y_val, (y_predicted_val>np.where(scores == np.max(scores))[0][0]/100), pos_label=0))

    recs = np.array(recs)
    best_rec_i = (np.abs(recs - target_rec)).argmin()
    best_rec = best_rec_i/1000

    print(recs[best_rec_i])

    from sklearn.metrics import classification_report
    from sklearn.metrics import accuracy_score, confusion_matrix
    print(classification_report(y_val, y_predicted_val>best_rec))
    print(confusion_matrix(y_val, y_predicted_val>best_rec))
    print(accuracy_score(y_val, y_predicted_val>best_rec))
    
    precision, recall, fscore, support = precision_recall_fscore_support(y_val, y_predicted_val>best_rec)
    
    return precision, recall, fscore, best_rec


In [ ]:
from sklearn.metrics import roc_curve, auc
import matplotlib.pyplot as plt

fpr = dict()
tpr = dict()
roc_auc = dict()
for i in range(1):
    fpr[i], tpr[i], _ = roc_curve(y_test[:, i], y_predicted[:, i])
    roc_auc[i] = auc(fpr[i], tpr[i])

print(roc_auc)
plt.figure()
lw = 2
plt.plot(fpr[0], tpr[0], color='darkorange',
         lw=lw, label='ROC curve (area = %0.2f)' % roc_auc[0])
plt.plot([0, 1], [0, 1], color='navy', lw=lw, linestyle='--')
plt.xlim([0.0, 1.0])
plt.ylim([0.0, 1.05])
plt.xlabel('False Positive Rate')
plt.ylabel('True Positive Rate')
plt.title('Receiver operating characteristic example')
plt.legend(loc="lower right")
plt.show()


#SAME BUT FOR VAL
fpr = dict()
tpr = dict()
roc_auc = dict()
for i in range(1):
    fpr[i], tpr[i], _ = roc_curve(y_val[:, i], y_predicted_val[:, i])
    roc_auc[i] = auc(fpr[i], tpr[i])

print(roc_auc)
plt.figure()
lw = 2
plt.plot(fpr[0], tpr[0], color='darkorange',
         lw=lw, label='ROC curve (area = %0.2f)' % roc_auc[0])
plt.plot([0, 1], [0, 1], color='navy', lw=lw, linestyle='--')
plt.xlim([0.0, 1.0])
plt.ylim([0.0, 1.05])
plt.xlabel('False Positive Rate')
plt.ylabel('True Positive Rate')
plt.title('Receiver operating characteristic example')
plt.legend(loc="lower right")
plt.show()

In [ ]:
# y_predicted_cath = model_1.predict(x_cath[:,:2048,:])

In [ ]:
#FOR TEST
from sklearn.metrics import f1_score

scores = []
for x in range(100):
    scores.append(f1_score(y_test, (y_predicted>x/100), pos_label=0))
print(np.max(scores), np.where(scores == np.max(scores))[0][0]/100)

scores = []
for x in range(100):
    scores.append(f1_score(y_test, (y_predicted>x/100), pos_label=1))
print(np.max(scores), np.where(scores == np.max(scores))[0][0]/100, f1_score(y_test, (y_predicted>np.where(scores == np.max(scores))[0][0]/100), pos_label=0))


from sklearn.metrics import classification_report
from sklearn.metrics import accuracy_score, confusion_matrix
print(classification_report(y_test, y_predicted>0.45))
print(confusion_matrix(y_test, y_predicted>0.45))
print(accuracy_score(y_test, y_predicted>0.45))


In [ ]:
from matplotlib import pyplot

def plot_me(ecg_in, limits = [-1500, 1500]):
    mdf = pd.DataFrame()
    for each in range(12):
        mdf['ecg2'] = ecg_in[1:1000,each].flatten()
        mdf['ecg2'].plot()
        pyplot.ylim(limits)
        pyplot.show()

In [ ]:
def plot_this_one(in_num):
    print(in_num)
    print('True     ', y_test[in_num])
    print('Prob of being 1', y_predicted[in_num])
    plot_me(x_test[in_num,:,:])

In [ ]:
def plot_these(in_list):
    for each in in_list:
        plot_this_one(each)

In [ ]:
plot_these([20, 248, 437, 428, 651, 923, 1118, 1317])

In [ ]:
plot_these([9274, 8819, 7350, 6208, 4995, 2669, 309, 137])

In [ ]:
#32 channels, 12 layers
from sklearn.metrics import roc_curve, auc
import matplotlib.pyplot as plt

fpr = dict()
tpr = dict()
roc_auc = dict()
for i in range(1):
    fpr[i], tpr[i], _ = roc_curve(y_test[:, i], y_predicted[:, i])
    roc_auc[i] = auc(fpr[i], tpr[i])
    
plt.figure()
lw = 2
plt.plot(fpr[0], tpr[0], color='darkorange',
         lw=lw, label='ROC curve (area = %0.2f)' % roc_auc[0])
plt.plot([0, 1], [0, 1], color='navy', lw=lw, linestyle='--')
plt.xlim([0.0, 1.0])
plt.ylim([0.0, 1.05])
plt.xlabel('False Positive Rate')
plt.ylabel('True Positive Rate')
plt.title('Receiver operating characteristic example')
plt.legend(loc="lower right")
plt.show()

In [ ]:
# Inverse Scaling
preds_raw = (min_max_scaler[1]-min_max_scaler[0])*y_predicted + min_max_scaler[0]
y_test_raw = (min_max_scaler[1]-min_max_scaler[0])*y_test + min_max_scaler[0]

for each in np.concatenate((y_predicted,y_test), axis=1):
    print(each)

In [ ]:
import numpy as np
for each in np.concatenate( (np.array(pred_raw).flatten().reshape(-1,1), y_test_raw[:14976]), axis=1):
    print(each)

In [ ]:
np.array(preds).flatten().shape

In [ ]:
for each in y_test:
    print(each)

In [ ]:
# np.where(abs(y_test-np.mean(y_test))<1e-4)
np.where(y_train == np.min(y_train))

In [ ]:
y_train[10895]

In [ ]:
#min
from matplotlib import pyplot
mdf = pd.DataFrame()
for each in range(12):
    mdf['ecg2'] = x_train[27512,1:1000,each].flatten()
    mdf['ecg2'].plot()
    pyplot.show()

In [ ]:
#max
from matplotlib import pyplot
mdf = pd.DataFrame()
for each in range(12):
    mdf['ecg2'] = x_train[7815,1:1000,each].flatten()
    mdf['ecg2'].plot()
    pyplot.show()

In [ ]:
pred_raw = (min_max_scaler[1]-min_max_scaler[0])*np.array(preds).reshape((-1,1)) + min_max_scaler[0]
y_test_raw = (min_max_scaler[1]-min_max_scaler[0])*y_test + min_max_scaler[0]

In [ ]:
mdf = pd.DataFrame()
mdf['pred_raw'] = np.array(y_test_raw).squeeze()
mdf['pred_raw'].describe()

In [ ]:
mdf = pd.DataFrame()
mdf['pred_raw'] = np.array(pred_raw).reshape((-1,1)).squeeze()
mdf['pred_raw'].describe()

In [ ]:
hyper_true = y_test_raw[:14592]>5.5
hyper_pred = np.array(pred_raw).reshape((-1,1))>5.5

print(np.array(hyper_true).shape)
print(np.array(hyper_pred).shape)

from sklearn.metrics import confusion_matrix
from sklearn.metrics import accuracy_score, classification_report

cm1 = confusion_matrix(hyper_true,hyper_pred)
print('Confusion Matrix : \n', cm1)
print('Accuracy', accuracy_score(hyper_true, hyper_pred))
print(classification_report(hyper_true, hyper_pred))

### **Everything Below is Old**

In [ ]:
pred = model.predict(x_test)

In [ ]:
pred_raw = min_max_scaler.inverse_transform(pred)
y_test_raw = min_max_scaler.inverse_transform(y_test)

In [ ]:
mdf = pd.DataFrame()
mdf['pred_raw'] = np.array(pred_raw).squeeze()
mdf['pred_raw'].describe()

In [ ]:
mdf['pred_raw'].hist()

In [ ]:
from sklearn.preprocessing import QuantileTransformer

stand_scaler = QuantileTransformer(n_quantiles=len(y_test))
y_test_stand = stand_scaler.fit_transform(y_test)

In [ ]:
mdf = pd.DataFrame()
mdf['pred_raw'] = np.array(y_test_raw).squeeze()
mdf['pred_raw'].describe()

In [ ]:
mdf['pred_raw'].hist(bins=15)

In [ ]:
sum(y_test_raw>5.5)/len(y_test_raw)

In [ ]:
mdf = pd.DataFrame()
mdf['pred_raw'] = np.array(pred_raw).squeeze()
mdf['pred_raw'].describe()

In [ ]:
mdf['pred_raw'].hist()